In [32]:
from multiprocessing import Pool
from functools import partial
from itertools import product

In [19]:
from contextlib import contextmanager

@contextmanager
def poolcontext(*args, **kwargs):
    pool = multiprocessing.Pool(*args, **kwargs)
    yield pool
    pool.terminate()

In [40]:
def f(x,y,z):
    return x*y*z

In [41]:
pool_n = len(l)
with poolcontext(processes=pool_n) as pool:
        results = pool.starmap(f, product([1,2,3],[1],[2]) )
print(results)

[2, 4, 6]


In [ ]:

with poolcontext(processes=pool_n) as pool:
    bodies = list(pool.map(partial(f,y=1), bodies))

In [ ]:
def integration(bodies,dt,years,tA=[],downsample=100,verbose=0):
    tA.remove(0)

    dt = dt
    t=0
    tA = tA


    sim_len = years*np.pi*1e7
    pool_n = len(bodies)
    while t < sim_len:
        #Update the position and momentum arrays for each body
        #for i in range(0,len(bodies)):
        with poolcontext(processes=pool_n) as pool:

            bodies = list(pool.map(partial( bodies[i].update_orbit_shape , c_body=bodies[i].center_body ), bodies))
            
#            bodies[i].update_orbit_shape(bodies[bodies[i].center_body])
            
            if bodies[i].get_prec ==1:
                get_precession(bodies[0],bodies[i],bodies[i+1],dt)
                
            #Append to arrays, based on downsample rate
            #Update arrays based on downsample rate
            if t != 0 and int(t/dt) % downsample == 0:
                tA.append(t)
                if verbose:
                    print("Iteration number: ",t/dt)
                    
                bodies[i].update_arrays(t)
                
        #Update the position vectors
        get_position_vectors(bodies,verbose=0)
        
        #Get the forces between the bodies
        #zero out the current forces:
        for i in range(0,len(bodies)):
            bodies[i].current_F = np.zeros(3)
            
        for i in range(0,len(bodies)):

            if verbose:
                print("Getting force vector on body ",i)
                
            for j in range(i,len(bodies)):
                if i != j:
                    if verbose:
                        print("\tby body ",j)
                        
                    F_j_on_i = G * bodies[i].mass * bodies[j].mass * bodies[i].r_hat[bodies[j].index] / (bodies[i].r_mag[bodies[j].index])**2
                    F_i_on_j = -F_j_on_i
                    
                    bodies[i].current_F = bodies[i].current_F + F_j_on_i
                    bodies[j].current_F = bodies[j].current_F + F_i_on_j
            if verbose:
                print("Current force on object ",bodies[i].index,": ",bodies[i].current_F)
            if i != 0:
                bodies[i].update_momentum(dt)
            
        for i in range(0,len(bodies)):
            bodies[i].update_pos(dt)

        t = t+dt

In [ ]:
def update_orbit_shape(self,center_body)
    if self.get_e:
        #ECCENTRICITY, APOAPSIS, PERIAPSIS, SEMIMAJOR
        #get the periapsis and apoapsis of the planet around its central body
        #If current point is farther than farthest point; new apoapsis
        if self.r_mag[self.center_body] > self.apoapsis_mag:
            self.r_apoapsis = self.pos - center_body.pos
            self.apoapsis_mag = np.linalg.norm(self.r_apoapsis)

        #If current point is closer than closest point; new periapsis
        if self.r_mag[self.center_body] < self.periapsis_mag:
            self.r_periapsis = self.pos - center_body.pos
            self.periapsis_mag = np.linalg.norm(self.r_periapsis)

        #Update the samimajor axis length!
        self.semimajor = (self.apoapsis_mag + self.periapsis_mag)/2 #average of the periapsis and apoapsis distance

        self.get_eccentricity(center_body)

In [2]:
def update_arrays(self,t):
    self.posA.append(self.pos)
    self.momentumA.append(self.momentum)
    
    #If this body is tracking inclination:
    if self.get_inclination:
        self.inclinationA.append(self.inclination) #in degrees

    #If this body is tracking obliquity:
    if self.get_obliq:
        self.obliquity = obliquity(t/(np.pi*1e7))
        self.obliq_A.append(self.obliquity)

    #If this body is tracking eccentricity and orbital shape:
    if self.get_e:
        self.eA.append(self.e)

    #If this body is tracking precession
    if self.get_prec:
        self.precessionA.append(self.precession)

In [ ]:

spec = [
    ('value', int32),               # a simple scalar field
    ('array', float32[:]),          # an array field
    
    ('index', int32)
    ('s' , int32) #point size on graph
    ('c' , int32) #color on graph

    #scalar quantities
    ('radius', float32)
    ('mass', float32)

    #vector quantities
    ('pos' ,float32[:])
    ('vel' ,float32[:])
    ('momentum ',float32[:])

    #Body's position vectors
    #These vectors point from this body to all other bodies in the system
    ('r' ,float32[:])
    ('r_hat' ,float32[:])
    ('r_mag' ,float32[:])

    #arrays to track vector quantities
    ('posA' ,float32[:])
    ('momentumA' ,float32[:])

    #Current force on body:
    ('current_F' ,float32[:])

    #Eccentricity and orbital shape
    ('e_vec' ,float32[:])
    ('center_body' , int32) #which other body in teh system I want the eccentricity relative to
    ('r_apoapsis' ,float32[:])
    ('apoapsis_mag' , float32)
    ('r_periapsis' ,float32[:])
    ('periapsis_mag' , float32)
    ('e' , float32)
    ('eA' ,float32[:])
    ('get_e' , int32)

    ('semimajor' , float32)

    #For some bodies that have useful orbital inclinations
    ('get_inclination' , int32)
    ('inclination' , float32)
    ('inclinationA' ,float32[:])

    #obliquity
    ('get_obliq' , int32)
    ('obliquity' , float32)
    ('obliq_A' ,float32[:])

    #precession
    ('get_prec' , int32)
    ('precession', float32) #in degrees
    ('precessionA' ,float32[:])
]

In [2]:
def integration(bodies,dt,years,verbose=0):

    BA = []
    projectionmag = []

    dt = 1e4
    t=0
    tA = []



    #plt.figure(figsize=[20,10])
    while t < 1*np.pi*1e7:

        #Update the position and momentum arrays for each body
        for i in range(0,len(bodies)):
            self.posA.append(self.pos)
            self.momentumA.append(self.momentum)
        
        #update baricenter array and time array
 #       BA.append(B)
        tA.append(t)

        #Update the position vectors
        get_position_vectors(bodies,verbose=0)
#         rbody1_to_body3 = body3Pos - body1.pos
#         rbody2_to_body3 = body3Pos - body2.pos
#         rbody1_to_body2 = body2.pos - body1.pos
#         rbody1_to_body3mag = np.sqrt(sum([x**2 for x in rbody1_to_body3]))
#         rbody2_to_body3mag = np.sqrt(sum([x**2 for x in rbody2_to_body3]))
#         rbody1_to_body2mag = np.sqrt(sum([x**2 for x in rbody1_to_body2]))

        #Baricenter vectors and Projection vectors-------------------
#         baricenter = np.array([0,0,0])
#         rbar_to_body2 = body2.pos - baricenter
#         rbar_to_body2hat = rbar_to_body2/np.linalg.norm(rbar_to_body2)
#         PROPLANE = np.cross(rbar_to_body2hat,np.array([0,0,1]))
#         PROPLANEmag = np.sqrt(sum([x**2 for x in PROPLANE]))

#         rbar_to_body1 = body1.pos - baricenter
#         rbar_to_body1hat = rbar_to_body1/np.linalg.norm(rbar_to_body1)
#         rbar_to_body3 = body3Pos - baricenter
#         rbar_to_body3hat = rbar_to_body3/np.linalg.norm(rbar_to_body3)

#         project_body1 = (np.dot(rbar_to_body1,PROPLANE)/PROPLANEmag)*PROPLANE
#         project_body3 = (np.dot(rbar_to_body3,PROPLANE)/PROPLANEmag)*PROPLANE
#         project_body1Massag = np.sqrt(sum([x**2 for x in project_body1]))
#         project_body3mag = np.sqrt(sum([x**2 for x in project_body3]))


#         rbody1Momentumroject_to_body2 = body2.pos - project_body1
#         rbody1Momentumroject_to_body2hat = rbody1Momentumroject_to_body2/np.linalg.norm(rbody1Momentumroject_to_body2)
#         rbody1Momentumroject_to_body2mag = np.sqrt(sum([x**2 for x in rbody1Momentumroject_to_body2]))

#         rbody3project_to_body2 = body2.pos - project_body3
#         rbody3project_to_body2hat = rbody3project_to_body2/np.linalg.norm(rbody3project_to_body2)
#         rbody3project_to_body2mag = np.sqrt(sum([x**2 for x in rbody3project_to_body2]))
        #------------------------------------------------------------

        #B = L/(4*np.pi*rbody1_to_body2mag**2)
    #     if((project_body1Massag+project_body3mag)<=(darkR+body1R) and rbody2_to_body3mag < rbody1_to_body2mag):
    #         B = 0
    #     else:
    #         B = L/(4*np.pi*rbody1_to_body2mag**2)

        #Get the forces between the bodies
        
        for i in range(0,len(bodies)):
            self.current_F = np.zeros(3)
            for j in range(i,len(bodies)):
                if i != j:
                    F_j_on_i = G * self.mass * bodies[j].mass * self.r_hat[bodies[j].index] / (self.r_mag[bodies[j].index])**2
                    F_i_on_j = -F_j_on_i
                    
                    self.current_F = self.current_F + F_j_on_i
                    bodies[j].current_F = bodies[j].current_F + F_i_on_j
            if verbose:
                print("Current force on object ",self.index,": ",self.current_F)
            self.update_momentum(dt)
            
        for i in range(0,len(bodies)):
            self.update_pos(dt)

        t = t+dt

#    BPlot = np.asarray(BA)